# 1. Import Necessary Libraries

In [74]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from src.data_preprocessing import preprocess_data_monthly
from src.model_training import train_model, evaluate_model, save_model
from src.model_inference import load_preprocessing_pipeline, load_model, preprocess_new_entry, predict_units

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 2. Load the Data

In [75]:
picked_id = 'CHO-MIL-31000'
df = pd.read_csv(f'./datasets/{picked_id}-Sales.csv')

# Display the first few rows
print("Raw Data:")
display(df.head())

# Print basic info about df columns before processing
print("\nDataFrame Columns and Sample Values:")
for col in df.columns:
    unique_vals = df[col].unique()
    num_unique = len(unique_vals)
    sample_val = unique_vals[0] if num_unique > 0 else None
    print(f"Column: {col}, Unique Values: {num_unique}, Example Value: {sample_val}")

Raw Data:


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Country/Region,City,State/Province,Postal Code,Division,Region,Product ID,Product Name,Sales,Units,Gross Profit,Cost
0,1133,US-2021-138100-CHO-MIL-31000,2021-09-15,2027-03-13,Standard Class,138100,United States,New York City,New York,10011,Chocolate,Atlantic,CHO-MIL-31000,Wonka Bar - Milk Chocolate,9.75,3,6.33,3.42
1,3396,US-2022-121391-CHO-MIL-31000,2022-10-04,2028-03-29,First Class,121391,United States,San Francisco,California,94109,Chocolate,Pacific,CHO-MIL-31000,Wonka Bar - Milk Chocolate,6.50,2,4.22,2.28
2,4377,US-2023-103982-CHO-MIL-31000,2023-03-03,2028-08-28,Standard Class,103982,United States,Round Rock,Texas,78664,Chocolate,Interior,CHO-MIL-31000,Wonka Bar - Milk Chocolate,9.75,3,6.33,3.42
3,387,US-2021-158064-CHO-MIL-31000,2021-04-21,2026-10-16,Standard Class,158064,United States,Los Angeles,California,90008,Chocolate,Pacific,CHO-MIL-31000,Wonka Bar - Milk Chocolate,16.25,5,10.55,5.70
4,1397,US-2021-130729-CHO-MIL-31000,2021-10-24,2027-04-21,Standard Class,130729,United States,Rancho Cucamonga,California,91730,Chocolate,Pacific,CHO-MIL-31000,Wonka Bar - Milk Chocolate,9.75,3,6.33,3.42



DataFrame Columns and Sample Values:
Column: Row ID, Unique Values: 2137, Example Value: 1133
Column: Order ID, Unique Values: 1768, Example Value: US-2021-138100-CHO-MIL-31000
Column: Order Date, Unique Values: 897, Example Value: 2021-09-15
Column: Ship Date, Unique Values: 971, Example Value: 2027-03-13
Column: Ship Mode, Unique Values: 4, Example Value: Standard Class
Column: Customer ID, Unique Values: 1745, Example Value: 138100
Column: Country/Region, Unique Values: 2, Example Value: United States
Column: City, Unique Values: 360, Example Value: New York City
Column: State/Province, Unique Values: 53, Example Value: New York
Column: Postal Code, Unique Values: 444, Example Value: 10011
Column: Division, Unique Values: 1, Example Value: Chocolate
Column: Region, Unique Values: 4, Example Value: Atlantic
Column: Product ID, Unique Values: 1, Example Value: CHO-MIL-31000
Column: Product Name, Unique Values: 1, Example Value: Wonka Bar - Milk Chocolate
Column: Sales, Unique Values:

# 3. Aggregate the Data Monthly

In [76]:
# Convert 'Order Date' to datetime
df['Order Date'] = pd.to_datetime(df['Order Date'], format='%Y-%m-%d', errors='coerce')
df = df.dropna(subset=['Order Date'])

# Keep only numeric columns that make sense to aggregate monthly
# For example, Units, Sales, Gross Profit, and Cost.
# Drop all non-numeric columns before resampling to avoid string concatenation issues.
numeric_cols = ['Units']
df = df[['Order Date'] + numeric_cols]

# Set Order Date as index for resampling
df.set_index('Order Date', inplace=True)

# Aggregate by month (end of month) - sum numeric values
monthly_df = df.resample('ME').sum().dropna()

# Add Year and Month columns from the DateTime index
monthly_df['Year'] = monthly_df.index.year
monthly_df['Month'] = monthly_df.index.month

# Reset index to move 'Order Date' back to a column if needed
monthly_df.reset_index(inplace=True)

# Now monthly_df only contains numeric aggregates and clean date-derived columns
display(monthly_df.head())

print(monthly_df)

# Print basic info about monthly_df columns before processing
print("\nDataFrame Columns and Sample Values:")
for col in monthly_df.columns:
    unique_vals = monthly_df[col].unique()
    num_unique = len(unique_vals)
    sample_val = unique_vals[0] if num_unique > 0 else None
    print(f"Column: {col}, Unique Values: {num_unique}, Example Value: {sample_val}")


,Order Date,Units,Year,Month
0,2021-01-31,91,2021,1
1,2021-02-28,43,2021,2
2,2021-03-31,126,2021,3
3,2021-04-30,151,2021,4
4,2021-05-31,112,2021,5


   Order Date  Units  Year  Month
0  2021-01-31     91  2021      1
1  2021-02-28     43  2021      2
2  2021-03-31    126  2021      3
3  2021-04-30    151  2021      4
4  2021-05-31    112  2021      5
5  2021-06-30    146  2021      6
6  2021-07-31    122  2021      7
7  2021-08-31    173  2021      8
8  2021-09-30    187  2021      9
9  2021-10-31     96  2021     10
10 2021-11-30    278  2021     11
11 2021-12-31    232  2021     12
12 2022-01-31     83  2022      1
13 2022-02-28     69  2022      2
14 2022-03-31    130  2022      3
15 2022-04-30     98  2022      4
16 2022-05-31    145  2022      5
17 2022-06-30    105  2022      6
18 2022-07-31    136  2022      7
19 2022-08-31    139  2022      8
20 2022-09-30    223  2022      9
21 2022-10-31    152  2022     10
22 2022-11-30    287  2022     11
23 2022-12-31    174  2022     12
24 2023-01-31     84  2023      1
25 2023-02-28     53  2023      2
26 2023-03-31    135  2023      3
27 2023-04-30    132  2023      4
28 2023-05-31 

# 4. Preprocess the Data

In [77]:
X, y, preprocessing_pipeline = preprocess_data_monthly(monthly_df, pipeline=None, fit_pipeline=True)

print("\nPreprocessed Features:")
display(X.head())


Preprocessed Features:


,Year,Month
0,-1.341641,-1.593255
1,-1.341641,-1.303572
2,-1.341641,-1.013890
3,-1.341641,-0.724207
4,-1.341641,-0.434524


# 5. Split the Data

In [78]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, random_state=42
)
X_cv, X_test, y_cv, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)

print(f"\nTraining set shape: {X_train.shape}")
print(f"CV set shape: {X_cv.shape}")
print(f"Testing set shape: {X_test.shape}")


Training set shape: (28, 2)
CV set shape: (10, 2)
Testing set shape: (10, 2)


# 6. Train Random Forest

In [79]:
rf_model = train_model(X_train, y_train, model_type='random_forest')
rf_cv_metrics = evaluate_model(rf_model, X_cv, y_cv)
rf_test_metrics = evaluate_model(rf_model, X_test, y_test)

print("\nRandom Forest CV Metrics:")
for metric, value in rf_cv_metrics.items():
    print(f"{metric}: {value:.4f}")

print("\nRandom Forest Test Metrics:")
for metric, value in rf_test_metrics.items():
    print(f"{metric}: {value:.4f}")


Random Forest CV Metrics:
Mean Absolute Error (MAE): 54.1810
Root Mean Squared Error (RMSE): 62.9410
R² Score: -1.9348

Random Forest Test Metrics:
Mean Absolute Error (MAE): 42.8320
Root Mean Squared Error (RMSE): 47.4240
R² Score: 0.3598


/opt/anaconda3/envs/uscandy/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/anaconda3/envs/uscandy/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


# 7. Train Linear Regression

In [80]:
lr_model = train_model(X_train, y_train, model_type='linear')
lr_cv_metrics = evaluate_model(lr_model, X_cv, y_cv)
lr_test_metrics = evaluate_model(lr_model, X_test, y_test)

print("\nLinear Regression CV Metrics:")
for metric, value in lr_cv_metrics.items():
    print(f"{metric}: {value:.4f}")

print("\nLinear Regression Test Metrics:")
for metric, value in lr_test_metrics.items():
    print(f"{metric}: {value:.4f}")


Linear Regression CV Metrics:
Mean Absolute Error (MAE): 51.5387
Root Mean Squared Error (RMSE): 63.9360
R² Score: -2.0284

Linear Regression Test Metrics:
Mean Absolute Error (MAE): 44.9385
Root Mean Squared Error (RMSE): 51.1513
R² Score: 0.2553


/opt/anaconda3/envs/uscandy/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/anaconda3/envs/uscandy/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


# 8. Save the preprocessing pipeline and models

In [81]:
preprocessing_pipeline_path = './models/monthly_preprocessing_pipeline.pkl'
with open(preprocessing_pipeline_path, 'wb') as f:
    pickle.dump(preprocessing_pipeline, f)

rf_model_path = './models/monthly_model_rf.pkl'
save_model(rf_model, rf_model_path)

lr_model_path = './models/monthly_model_lr.pkl'
save_model(lr_model, lr_model_path)

print("\nPreprocessing pipeline, Linear Regression model and Random Forest model have been saved.")


Preprocessing pipeline, Linear Regression model and Random Forest model have been saved.


# 9. Optionally make predictions on new monthly data

In [82]:
# Suppose you have a new_data.csv file aggregated monthly:
new_data_path = './datasets/monthly_new_data.csv'
new_df = pd.read_csv(new_data_path)
X_new, _, _ = preprocess_data_monthly(new_df, pipeline=preprocessing_pipeline, fit_pipeline=False)
model_loaded = load_model(lr_model_path)
predictions = model_loaded.predict(X_new)
new_df['Predicted Units'] = predictions

print("\nNew Data with Predictions:")
display(new_df)

output_path = 'models/monthly_predictions_monthly.csv'
new_df.to_csv(output_path, index=False)
print(f"\nPredictions have been saved to '{output_path}'.")


New Data with Predictions:


,Order Date,Year,Month,Predicted Units
0,2021-06-30,2021,6,133.394653
1,2021-07-31,2021,7,149.211672
2,2021-08-31,2021,8,165.028690
3,2021-09-30,2021,9,180.845709



Predictions have been saved to 'models/monthly_predictions_monthly.csv'.
